In [29]:
import pandas as pd
import os
from glob import glob
import operator as op


In [38]:
fnames = glob('../data/raw/*.csv')
df = pd.concat([pd.read_csv(fname) for fname in fnames], axis=0, ignore_index=True)
df.head()

,1080P Input Lag,3D Projector,Aspect Ratio,Audible Noise,Brightness,Contrast,L × W × H,Lamp Life,Resolution,Screen Size,...,Technology,Throw Distance,Throw Ratio (std lens),User Rating,Volume,Warranty,Weight,name,price_dollars,release_date
0,44 ms,No,16:10,36 dB,"3,300 lumens","6,000:1",26 × 34 × 11 cm,"4,000 hours",1280x800 (WXGA),76 –762 cm,...,1 cm3 LCD,1.4 – 8.4 m,1.30– 2.20,Rate this Projector,9.56 L,3 years,2.9 kg,NEC ME331W,$639,1/1/2017
1,16 ms,No,16:9,n/a,"4,000 lumens","12,000:1",26 × 33 × 13 cm,"4,000 hours",1920x1080 (HD),76 –763 cm,...,2 cmDLP,1.2 – 9.9 m,1.49– 1.76,Rate this Projector,11.69 L,3 years,3.7 kg,ViewSonic PG705HD,$799,12/20/2017
2,32 ms,Full HD 3D,1.9:1,n/a,"1,500 lumens",n/a,46 × 50 × 20 cm,n/a,4096x2160,151 –407 cm,...,2 cmSXRD,3.8 – 5.0 m,1.38– 2.83,Rate this Projector,44.85 L,3 years,14.1 kg,*,"$7,999",First Ship Date:
3,33 ms,No,16:9,n/a,300 lumens,"100,000:1",9 × 11 × 6 cm,"30,000 hours",1280x720 (HD),51 –254 cm,...,DLP,0.7 – 3.8 m,1.71,NaN,0.61 L,2 years,0.4 kg,*,$450,First Ship Date:
4,16 ms,Full HD 3D,4:3,n/a,"3,500 lumens","22,000:1",21 × 32 × 12 cm,"4,500 hours",800x600 (SVGA),75 –762 cm,...,DLP,1.3 –12.0 m,1.97– 2.17,Rate this Projector,8.24 L,3 years,2.1 kg,ViewSonic PA502S,$289,7/15/2017


In [78]:
a = 'adfafHH'
a.isupper()

False

In [80]:
import string
str.isupper

AttributeError: module 'string' has no attribute 'isupper'

In [81]:
dfn = pd.DataFrame()
dfn['Name'] = df['name']
dfn['Input Lag'] = df['1080P Input Lag'].apply(lambda s: int(s.split(' ')[0]))
dfn['Aspect Ratio'] = df['Aspect Ratio'].apply(lambda s: op.truediv(*[float(a) for a in s.split(':')]))
dfn['Audible Noise'] = df['Audible Noise'].str.replace('n/a', '0 dB').str.replace(' dB', '').astype(int)
dfn.loc[dfn['Audible Noise'] == 0, 'Audible Noise'] = None
dfn['Brightness'] = df['Brightness'].apply(lambda s: int(s.split(' ')[0].replace(',', '')))
dfn['Contrast'] = df['Contrast'].str.replace('n/a', '0:1').apply(lambda s: s.split(':')[0].replace(',', '')).astype(int)
dfn.loc[dfn['Contrast'] == 0, 'Contrast'] = None
dfn['Length'] = df['L × W × H'].apply(lambda s: int(s.split('×')[0]))
dfn['Width'] = df['L × W × H'].apply(lambda s: int(s.split('×')[1]))
dfn['Height'] = df['L × W × H'].apply(lambda s: int(s.split('×')[2].replace('cm', '')))
dfn['Resolution Width'] = df['Resolution'].apply(lambda s: int(s.split('x')[0]))
dfn['Resolution Height'] = df['Resolution'].apply(lambda s: int(s.split('x')[1].split('(')[0]))
dfn['Volume'] = df['Volume'].apply(lambda s: float(s.replace('L', '')))
dfn['Weight'] = df['Weight'].apply(lambda s: float(s.replace('kg', '')))
dfn['Price'] = df['price_dollars'].apply(lambda s: int(s.replace('n/a', '0').replace('$', '').replace(',', '')))
dfn.loc[dfn['Price'] == 0, 'Price'] = None
dfn['Release Date'] = df['release_date']
dfn['3D'] = df['3D Projector'].str.contains('3D')
dfn.loc[dfn['Release Date'].str.contains('First'), 'Release Date'] = None
dfn['Technology'] = df['Technology'].apply(lambda s: ''.join(filter(str.isupper, s)))
dfn.head()

,Name,Input Lag,Aspect Ratio,Audible Noise,Brightness,Contrast,Length,Width,Height,Resolution Width,Resolution Height,Volume,Weight,Price,Release Date,3D,Technology
0,NEC ME331W,44,1.600000,36.0,3300,6000.0,26,34,11,1280,800,9.56,2.9,639.0,1/1/2017,False,LCD
1,ViewSonic PG705HD,16,1.777778,NaN,4000,12000.0,26,33,13,1920,1080,11.69,3.7,799.0,12/20/2017,False,DLP
2,*,32,1.900000,NaN,1500,NaN,46,50,20,4096,2160,44.85,14.1,7999.0,None,True,SXRD
3,*,33,1.777778,NaN,300,100000.0,9,11,6,1280,720,0.61,0.4,450.0,None,False,DLP
4,ViewSonic PA502S,16,1.333333,NaN,3500,22000.0,21,32,12,800,600,8.24,2.1,289.0,7/15/2017,True,DLP


In [82]:
dfn.to_csv('../data/final/projectors.csv', index=False)